---
[Philipp Schreiber](https://github.com/pcschreiber1)

# Simulation Study of spatial Average Treamtent Effect (ATE) estimation
## For the replication of Henderson, Storeygard, Deichmann (2017)

---

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import statsmodels.formula.api as smf
import seaborn as sns

#For spatial analysis
import geopandas as gpd
import shapely.geometry as geom
import libpysal as lp #For spatial weights

from pysal.viz import splot #exploratory analysis
#from splot.esda import plot_moran #exploratory analysis
from pysal.explore import esda #exploratory analysis
from pysal.model import spreg #For spatial regression

pd.options.display.float_format = "{:,.2f}".format

In [3]:
from auxiliary.data_import import *
from auxiliary.plots import *
from auxiliary.simulations import *

**Three simulation studies:**

 - SLX specification: $Y = WD\gamma +  X \beta + u $ where $W$ is the weight matrix and $\gamma=0.25$, $\beta = 0.5$
 
 - SDM specification: $Y = WY \rho +   WD\gamma +  X \beta + u $ where $W$ is the weight matrix and $\gamma=0.25$, $\beta = 0.5$, $\rho = 0.05$.
 
 - Spatial Lag specification: $Y = WY \rho +   D\gamma +  X \beta + u $ where $W$ is the weight matrix and $\gamma=0.25$, $\beta = 0.5$, $\rho = 0.05$.
 
 - Backdoor specification: $Y = WY \rho +   ??D\gamma +  X \beta + u $ and $D = WD$ where $W$ is the weight matrix and $\gamma=0.25$, $\beta = 0.5$, $\rho = 0.05$.
 
We compare true ATE, non-spatial estimate and spatial estimate for three different sample sizes:
   - Simple: One simulation with $n=100$
   - Small: $100$ simulations with $n=100$
   - Large: $100$ simulations with $n=2.500$ (for SLX $100$ simulations with $n=10.000$)
   
Further details: the weight matrix $W$ is generated using a knn=10 nearest neighbour approach.

**SLX Simulation**

In [ ]:
# SLX sample
def simulate_SLX_sample(num_obs,
                        knn = 10,
                        beta = 0.9,
                        gamma = 0.25):
    """Simulate spatial sample with only spillover from treatment variable
        "Y = WD + X"

    Args:
        num_obs: An integer that specifies the number of individuals
            to sample. Needs to be either 100 or 10.000 for the weight matrix.

    Returns:
        Returns a dataframe with the observables (Y, X, D) as well as
        the unobservables (Y_1, Y_0) and the weight matrix w.
    """

    # Initialize empty data frame
    columns = ["Y", "Y_1", "Y_0", "D", "X", "WD"]
    df = pd.DataFrame(columns=columns, index=range(num_obs))

    df["D"] = np.random.randint(2, size=num_obs) #binary treatment
    df["X"] = np.random.normal(size=num_obs)
    
    if num_obs == 100:
            # generate grid for weight matrix
            x,y=np.indices((10, 10))
            x.shape=(num_obs,1)
            y.shape=(num_obs,1)
            data=np.hstack([x,y])
    elif num_obs == 10000:
            # generate grid for weight matrix
            x,y=np.indices((100, 100))
            x.shape=(num_obs,1)
            y.shape=(num_obs,1)
            data=np.hstack([x,y])
    else:
            raise AssertionError # needs to be 100 or 10.000 for constructing weight matrix

    # weight matrix
    w = lp.weights.KNN(data, k = knn)
    
    # calculate spillovers
    df["WD"] = np.dot(w.full()[0], df["D"].to_numpy())#not standardized
    
    # outcomes
    df["Y"] = beta*df["X"] + gamma*df["WD"] + gamma*df["D"] #add D since W is sparse
    df["Y_1"] = beta*df["X"] + gamma*df["WD"] + gamma
    df["Y_0"] = beta*df["X"] + gamma*df["WD"]
    
    return df, w

In [94]:
np.random.seed(2021)

# define the number of simulations and number of observations to be simulated
n_sims = [1, 100, 100]
n_obs = [100, 100, 10000]

# initialize the container
columns = ["Sim1", "Sim2", "Sim3"]
df = pd.DataFrame(columns=columns, index=["ATE", "Non-spatial", "spatial"])


for _, n in enumerate(n_sims):
    
    #initialize containers
    ATE = np.empty((n_sims[_],1))
    Nspat = np.empty((n_sims[_],1))
    Spat = np.empty((n_sims[_],1))
    
    for j in range(0, n):
        data, w = simulate_SLX_sample(n_obs[_])
        # calculate values
        ate_true = data["Y_1"].sub(data["Y_0"]).mean()
        nonspatial_ols = smf.ols("Y ~ X + D", data=data).fit().params[2]
        spatial_ols = smf.ols("Y ~ X + D + WD", data=data).fit().params[2]
                   
        
        #store in container
        ATE[j] = ate_true
        Nspat[j] = nonspatial_ols
        Spat[j] = spatial_ols

    #save in dataframe
    df.loc[:, columns[_]] = [np.mean(ATE), np.mean(Nspat), np.mean(Spat)]


In [95]:
df.head()

,Sim1,Sim2,Sim3
ATE,0.25,0.25,0.25
Non-spatial,0.55,0.24,0.24
spatial,0.25,0.25,0.25


In [41]:
# df.to_csv("data/SLX_sim.csv") #current file is with 10000 simulations

**SDM Simulation**

In [58]:
np.random.seed(123)

n_sims = [1, 100, 100]
n_obs = [100, 100, 2500]

columns = ["Sim1", "Sim2", "Sim3"]
df = pd.DataFrame(columns=columns, index=["ATE", "Non-spatial", "spatial"])


for _, n in enumerate(n_sims):
    
    #initialize containers
    ATE = np.empty((n_sims[_],1))
    Nspat = np.empty((n_sims[_],1))
    Spat = np.empty((n_sims[_],1))
    
    for j in range(0, n):
        data, w = simulate_SDM_sample(n_obs[_])
        # calculate values
        ate_true = data["Y_1"].sub(data["Y_0"]).mean()
        nonspatial_ols = smf.ols("Y ~ X + D", data=data).fit().params[2]
        #spatial 2 stage
        # preparing data for pysal spreg
        y = data["Y"].to_numpy()
        y = np.reshape(y, (y.size, 1))
        
        X = []
        X.append(data["X"].to_numpy())
        X.append(data["D"].to_numpy())
        X.append(data["WD"].to_numpy())
        X = np.array(X).T
        
        #row standardize matrix
        w.transform = 'r'
        
        #two-stage regression
        reg = spreg.GM_Lag(y, X, w=w,w_lags=1, name_x=['X', 'D', 'WD'], name_y='Y',name_ds='simulation')
        spatial_2stage = reg.betas[2][0]
        
        #store in container
        ATE[j] = ate_true
        Nspat[j] = nonspatial_ols
        Spat[j] = spatial_2stage

    #save in dataframe
    df.loc[:, columns[_]] = [np.mean(ATE), np.mean(Nspat), np.mean(Spat)]

In [59]:
df.head()

,Sim1,Sim2,Sim3
ATE,0.25,0.25,0.25
Non-spatial,0.36,0.34,0.41
spatial,0.25,0.25,0.25


In [60]:
df.to_csv("data/SDM_sim.csv")

**Backdoor Specification**

In [140]:
np.random.seed(123)

n_sims = [1, 100, 100]
n_obs = [100, 100, 2500]

columns = ["Sim1", "Sim2", "Sim3"]
df = pd.DataFrame(columns=columns, index=["ATE", "Non-spatial", "spatial"])


for _, n in enumerate(n_sims):
    
    #initialize containers
    ATE = np.empty((n_sims[_],1))
    Nspat = np.empty((n_sims[_],1))
    Spat = np.empty((n_sims[_],1))
    
    for j in range(0, n):
        data, w = simulate_backdoor_sample(n_obs[_])
        # calculate values
        ate_true = data["Y_1"].sub(data["Y_0"]).mean()
        nonspatial_ols = smf.ols("Y ~ X + D", data=data).fit().params[2]
        #spatial 2 stage
        # preparing data for pysal spreg
        y = data["Y"].to_numpy()
        y = np.reshape(y, (y.size, 1))
        
        X = []
        X.append(data["X"].to_numpy())
        X.append(data["D"].to_numpy())
        X.append(data["WD"].to_numpy())
        X = np.array(X).T
        
        #row standardize matrix
        w.transform = 'r'
        
        #two-stage regression
        reg = spreg.GM_Lag(y, X, w=w,w_lags=1, name_x=['X', 'D', 'WD'], name_y='Y',name_ds='simulation')
        spatial_2stage = reg.betas[2][0]
        
        #store in container
        ATE[j] = ate_true
        Nspat[j] = nonspatial_ols
        Spat[j] = spatial_2stage

    #save in dataframe
    df.loc[:, columns[_]] = [np.mean(ATE), np.mean(Nspat), np.mean(Spat)]

In [141]:
df.head()

,Sim1,Sim2,Sim3
ATE,0.25,0.25,0.25
Non-spatial,0.30,0.41,0.41
spatial,0.25,0.25,0.25


In [142]:
df.to_csv("data/backdoor_sim.csv")

In [9]:
#Spatial Lag sample
def simulate_SpatialLag_sample(num_obs,
                            knn = 10,
                            beta = 0.9,
                            gamma = 0.25,
                            rho = 0.05):
    """Simulate spatial sample with spillover from the treatment and the outcome variable
        "Y = WY+ WD + X"

    Args:
        num_obs: An integer that specifies the number of individuals
            to sample.

    Returns:
        Returns a dataframe with the observables (Y, X, D) as well as
        the unobservables (Y_1, Y_0)and the weight matrix w.
    """

    # Initialize empty data frame
    columns = ["Y", "Y_1", "Y_0", "D", "X", "WD"]
    df = pd.DataFrame(columns=columns, index=range(num_obs))

    df["D"] = np.random.randint(2, size=num_obs) #binary treatment
    df["X"] = np.random.normal(size=num_obs)
    
    if num_obs == 100:
            # generate grid for weight matrix
            x,y=np.indices((10, 10))
            x.shape=(num_obs,1)
            y.shape=(num_obs,1)
            data=np.hstack([x,y])
    elif num_obs == 2500:
            # generate grid for weight matrix
            x,y=np.indices((50, 50))
            x.shape=(num_obs,1)
            y.shape=(num_obs,1)
            data=np.hstack([x,y])
    else:
            raise AssertionError # needs to be 100 or 10.000 for constructing weight matrix

    # weight matrix
    w = lp.weights.KNN(data, k = knn)
    
    # calculate spillovers
    df["WD"] = np.dot(w.full()[0], df["D"].to_numpy())#not standardized
    
    # outcomes
    df["Y"] = beta*df["X"] + gamma*df["D"] #add D since W is sparse
    df["Y_1"] = beta*df["X"] + gamma
    df["Y_0"] = beta*df["X"] 
    
    df["Y_no_spill"] = df["Y"]
    df["Y_1_no_spill"] = df["Y_1"]
    df["Y_0_no_spill"] = df["Y_0"]
    
    # iterate to generate general equilibrium effect (already settles after 5 times for small rho)
    for i in range(0,10):
        df["WY"] = np.dot(w.full()[0], df["Y"].to_numpy())#not standardized
        df["Y"] = beta*df["X"] + gamma*df["WD"] + gamma*df["D"] + rho* df["WY"]   
        df["Y_1"] = beta*df["X"] + gamma*df["WD"] + gamma + rho* df["WY"]   
        df["Y_0"] = beta*df["X"] + gamma*df["WD"] + rho* df["WY"]   
    
    return df, w


In [16]:
# Spatial lag model

np.random.seed(123)

n_sims = [1, 100, 100]
n_obs = [100, 100, 2500]

columns = ["Sim1", "Sim2", "Sim3"]
df = pd.DataFrame(columns=columns, index=["ATE", "Non-spatial", "spatial"])


for _, n in enumerate(n_sims):
    
    #initialize containers
    ATE = np.empty((n_sims[_],1))
    Nspat = np.empty((n_sims[_],1))
    Spat = np.empty((n_sims[_],1))
    
    for j in range(0, n):
        data, w = simulate_SpatialLag_sample(n_obs[_])
        # calculate values
        ate_true = data["Y_1"].sub(data["Y_0"]).mean()
        nonspatial_ols = smf.ols("Y ~ X + D", data=data).fit().params[2]
        #spatial 2 stage
        # preparing data for pysal spreg
        y = data["Y"].to_numpy()
        y = np.reshape(y, (y.size, 1))
        
        X = []
        X.append(data["X"].to_numpy())
        X.append(data["D"].to_numpy())
        X = np.array(X).T
        
        #row standardize matrix
        w.transform = 'r'
        
        #two-stage regression
        reg = spreg.ML_Lag(y, X, w=w,name_x=['X', 'D'], epsilon=1e-07, name_y='Y',name_ds='simulation')
        spatial_2stage = reg.betas[2][0]
        
        #store in container
        ATE[j] = ate_true
        Nspat[j] = nonspatial_ols
        Spat[j] = spatial_2stage

    #save in dataframe
    df.loc[:, columns[_]] = [np.mean(ATE), np.mean(Nspat), np.mean(Spat)]

In [17]:
df.to_csv("data/Spatial_Lag_sim.csv")

In [18]:
df.head()

,Sim1,Sim2,Sim3
ATE,0.25,0.25,0.25
Non-spatial,0.36,0.34,0.41
spatial,0.13,0.12,0.14


In [5]:
reg = spreg.GM_Lag(y, X, w=w,w_lags=1, name_x=['X', 'D', 'WD'], name_y='Y',name_ds='simulation')

In [7]:
print(reg.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :  simulation
Weights matrix      :     unknown
Dependent Variable  :           Y                Number of Observations:         100
Mean dependent var  :      0.5100                Number of Variables   :           5
S.D. dependent var  :      1.0243                Degrees of Freedom    :          95
Pseudo R-squared    :      1.0000
Spatial Pseudo R-squared:  1.0000

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -0.0000873       0.0000068     -12.8074509       0.0000000
                   X       0.8999995       0.0000014    642640.0141976       0.0000000
                   D       0.2500032       0.0000056    4